In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
PROJECT_ROOT = os.path.abspath("..")  # adjust if needed
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
from scripts.trainer import ChestXrayHandler
import yaml
import os
import pandas as pd

/usr/local/pace-apps/manual/packages/anaconda3/2023.03/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def fix_path(p, config_path="../configs/config_HCV.yaml"):
    if p is None:
        return None
    if isinstance(p, str) and not os.path.isabs(p):
        base = os.path.dirname(os.path.abspath(config_path))
        return os.path.abspath(os.path.join(base, p))
    return p

In [4]:
with open("../configs/config_HCV.yaml", "r") as f:
    raw_cfg = yaml.safe_load(f)
raw_cfg

{'test_model': {'value': False,
  'help': 'true to test the model, false to train'},
 'epochs': {'value': 2, 'help': 'Number of epochs to train.'},
 'batch_size': {'value': 16, 'help': 'Batch size for training.'},
 'lr': {'value': 0.0001, 'help': 'Learning rate.'},
 'lr_decay': {'value': 0.4, 'help': 'Gamma value for StepLR scheduler.'},
 'train_last_vit_layers': {'value': 2,
  'help': 'Train only the last N transformer layers in ViT.'},
 'workers': {'value': 4, 'help': 'Number of dataloader workers.'},
 'device': {'value': 'auto', 'help': 'Training device: auto/cpu/cuda/mps.'},
 'freeze_backbone': {'value': True, 'help': 'Freeze DenseNet backbone.'},
 'freeze_vit': {'value': True, 'help': 'Freeze ViT encoder.'},
 'use_custom_proj': {'value': False,
  'help': 'Use custom DenseNet→ViT projection block.'},
 'img_dir': {'value': '../nih_data', 'help': 'Directory of image folders.'},
 'csv_path': {'value': '../nih_data/Data_Entry_2017_Sample.csv',
  'help': 'Path to CSV file.'},
 'save_dir

In [5]:
class Dict2Obj:
    def __init__(self, d):
        for key, item in d.items():
            # YAML format: key: {value: X, help: Y}
            if isinstance(item, dict) and "value" in item:
                setattr(self, key, item["value"])
            else:
                setattr(self, key, item)
args = Dict2Obj(raw_cfg)

args.img_dir  = fix_path(args.img_dir)
args.csv_path = fix_path(args.csv_path)
args.save_dir = fix_path(args.save_dir)
if args.resume:
    args.resume = fix_path(args.resume)

# prevent deadlocks
args.workers = 0

# override training hyperparameters - If you want to change them only in the config, comment this out!
args.epochs = 3
args.batch_size = 8
args.lr = 0.0001
args.verbose = True
# disable resume
args.resume = None


In [ ]:
trainer = ChestXrayHandler(args)

In [59]:
trainer.run()

Epoch 1 Training: 100%|██████████| 23/23 [00:02<00:00, 10.63it/s, loss=0.0961]


Train Loss: 0.1665 | Train Acc: 0.7211


Epoch 1 Validation: 100%|██████████| 8/8 [00:00<00:00, 11.67it/s]


Val Loss=0.1145 | Val Acc=0.8384 | Macro F1=0.2128
Saved BEST model → Macro F1: 0.2128


Epoch 2 Training: 100%|██████████| 23/23 [00:02<00:00, 10.16it/s, loss=0.0659]


...

Val Loss=0.1035 | Val Acc=0.8607 | Macro F1=0.2092


Epoch 3 Training: 100%|██████████| 23/23 [00:02<00:00,  9.91it/s, loss=0.0785]


Train Loss: 0.1077 | Train Acc: 0.8668


Epoch 3 Validation: 100%|██████████| 8/8 [00:00<00:00, 10.92it/s]


Val Loss=0.1041 | Val Acc=0.8677 | Macro F1=0.1950


In [ ]:
result_3 = pd.read_csv("../results/training_results.csv")
result_3.head()

In [ ]:
trainer.test()

In [67]:
# check the result
result_1 = pd.read_csv("../results/final_test_per_class.csv")
result_1.head()

,Disease,F1,AUC
0,Atelectasis,0.000000,0.487943
1,Cardiomegaly,0.272727,0.702381
2,Consolidation,0.203390,0.548052
3,Edema,0.000000,0.636905
4,Effusion,0.406250,0.551773


In [68]:
result_2 = pd.read_csv("../results/final_test_summary.csv")
result_2.head()

,metric,value
0,test_loss,0.110137
1,test_accuracy,0.413594
2,macro_f1,NaN


In [ ]:
#### later we can create some plots. give more analysis.